In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install mido
import numpy as np
import tensorflow as tf
#physical_devices = tf.config.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
from tensorflow.keras.layers import Input, LSTM, Dropout, Dense, Flatten
from tensorflow.keras.models import Model

     |████████████████████████████████| 61kB 4.9MB/s 


In [ ]:
from mido import MidiFile
note_duration = []
b_p_m = 120
output_all = []
bpm = []

def conversion_to_list(midi_path):
    mid = MidiFile(midi_path)
    mididict = []
    output = []

    # Put all note on/off in midinote as dictionary.
    for i in mid:
        if i.type == 'note_on' or i.type == 'note_off' or i.type == 'time_signature':
            mididict.append(i.dict())
        if i.is_meta and i.type == "set_tempo":
            tempo = i.tempo
            b_p_m = mido.tempo2bpm(tempo)
            bpm.append(b_p_m)
    # change time values from delta to relative time.
    mem1=0
    for i in mididict:
        time = i['time'] + mem1
        i['time'] = time
        mem1 = i['time']
    # make every note_on with 0 velocity note_off
        if i['type'] == 'note_on' and i['velocity'] == 0:
            i['type'] = 'note_off'
    # put note, starttime, stoptime, as nested list in a list. # format is [type, note, time, channel]
        mem2=[]
        if i['type'] == 'note_on' or i['type'] == 'note_off':
            mem2.append(i['type'])
            mem2.append(i['note'])
            mem2.append(i['time'])
            mem2.append(i['channel'])
            output.append(mem2)
            output_all.append(mem2)
    # put timesignatures
        #if i['type'] == 'time_signature':
            #mem2.append(i['type'])
            #mem2.append(i['numerator'])
            #mem2.append(i['denominator'])
            #mem2.append(i['time'])
            #output.append(mem2)
            #output_all.append(mem2)
    # viewing the midimessages.
    #for i in output:
        #print(i)
    #print(mid.ticks_per_beat)
    return output

In [ ]:
def duration_conversion(duration):
    if duration < 0.125:
        duration = 0.125
    elif duration >= 0.125 and duration < 0.375:
        duration = 0.25
    elif duration >= 0.375 and duration < 0.625:
        duration = 0.5
    elif duration >= 0.625 and duration < 0.875:
        duration = 0.75
    elif duration >= 0.875 and duration < 1.25:
        duration = 1
    elif duration >= 1.25 and duration < 1.75:
        duration = 1.5
    elif duration >= 1.75 and duration < 2.5:
        duration = 2
    elif duration >= 2.5 and duration < 3.5:
        duration = 3
    elif duration >= 3.5 and duration <= 5:
        duration = 4
    elif duration > 5 and duration <= 7:
        duration = 6
    elif duration > 7 and duration <= 12:
        duration = 8
    elif duration > 12 and duration <= 24:
        duration = 16
    else:
        duration = 32
    return duration

In [ ]:
def rest_conversion(rest):
    if rest < 0.05:
        rest = 0
    elif rest >= 0.05 and rest < 0.1:
        rest = 0.1
    elif rest >= 0.1 and rest < 0.5:
        rest = 0.25
    elif rest >= 0.5 and rest <= 1.5:
        rest = 1
    elif rest >= 1.5 and rest <= 3:
        rest = 2
    elif rest >= 3 and rest <= 6:
        rest = 4
    elif rest >= 6 and rest <= 12:
        rest = 8
    elif rest >= 12 and rest <= 24:
        rest = 16
    else:
        rest = 32
    return rest

In [ ]:
# def rest_conversion(duration):
#     if duration < 0.125:
#         duration = 0.125
#     elif duration >= 0.125 and duration < 0.375:
#         duration = 0.25
#     elif duration >= 0.375 and duration < 0.625:
#         duration = 0.5
#     elif duration >= 0.625 and duration < 0.875:
#         duration = 0.75
#     elif duration >= 0.875 and duration < 1.25:
#         duration = 1
#     elif duration >= 1.25 and duration < 1.75:
#         duration = 1.5
#     elif duration >= 1.75 and duration < 2.5:
#         duration = 2
#     elif duration >= 2.5 and duration < 3.5:
#         duration = 3
#     elif duration >= 3.5 and duration <= 5:
#         duration = 4
#     elif duration > 5 and duration <= 7:
#         duration = 6
#     elif duration > 7 and duration <= 12:
#         duration = 8
#     elif duration > 12 and duration <= 24:
#         duration = 16
#     else:
#         duration = 32
#     return duration

In [ ]:
def calculate_note_duration(output):
  for i, o in enumerate(output):
      if o[0] == "note_on":
          note = o[1]
          start_time = output[i-1][2]
          if i == 0:
            start_time = 0
          for idx, s in enumerate(output):
              if idx > i and s[1] == note and s[0] == "note_off":
                  stop_time = output[idx-1][2]
                  duration = stop_time - start_time
                  temp = []
                  temp.append(note)
                  duration = duration * b_p_m / 60
                  duration = duration_conversion(duration)
                  temp.append(duration)
                  note_duration.append(temp)
                  #print(idx)
                  break

In [ ]:
start_time = []
def calculate_start_time():
  for i, o in enumerate(output_all):
      if o[0] == "note_on":
          start_time.append(output_all[i-1][2])
  start_time[0] = 0

In [ ]:
rest_time = []
avg_bpm = np.mean(bpm)
def calculate_rest_time():
  for idx in range(len(note_duration) - 1):
      rest = start_time[idx + 1] - start_time[idx]
      rest = rest * avg_bpm / 60
      rest = rest_conversion(rest)
      rest_time.append(rest)
  rest_time.append(0)

In [ ]:
def append_rest_time():  
  for idx, a in enumerate(note_duration):
    a.append(rest_time[idx])

In [ ]:
import glob
import mido
from mido import MidiFile, MidiTrack, Message
count = 0
for folder in glob.glob('/content/drive/MyDrive/Major Project/datasets/classical_midi/*'):
  for midi in glob.glob(folder + '/*.mid'):
    #print(midi)
    print(count)

    output = conversion_to_list(midi)
    calculate_note_duration(output)
    
    count += 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
np.save("/content/drive/MyDrive/Major Project/paper/note_duration.npy", np.array(note_duration))
np.save("/content/drive/MyDrive/Major Project/paper/output_all.npy", np.array(output_all))

In [ ]:
output_all[0]

['note_on', 82, 0, 0]

In [ ]:
calculate_start_time()

In [ ]:
calculate_rest_time()
append_rest_time()

In [ ]:
print(len(output_all))
print(len(start_time))
print(len(rest_time))
print(len(note_duration))

1521514
760755
760755
760755


In [ ]:
note_duration[100:110]

[[72, 0.125, 0.125],
 [64, 0.25, 0.25],
 [81, 0.25, 0.125],
 [72, 0.125, 0.125],
 [53, 0.25, 0.125],
 [65, 0.25, 0.25],
 [81, 0.25, 0.125],
 [72, 0.125, 0.125],
 [55, 0.25, 0.25],
 [72, 0.25, 0.25]]

In [ ]:
np.array(note_duration).shape

(760755, 3)

In [ ]:
np.save("/content/drive/MyDrive/Major Project/paper/note_duration_final.npy", np.array(note_duration))